In [ ]:
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Get RPM values by file_id

import glob
files = glob.glob("/home/marcelo/Documents/UTEM/Research/cancer/gdc_download_20210427_234814.548111/*/*.mirbase21.mirnas.quantification.txt")
RPM={}
for file in files:
    data = pd.read_csv(file, sep='\t', lineterminator='\n')
    rpm = data[ data["miRNA_ID"]=="hsa-mir-149" ]["reads_per_million_miRNA_mapped"].values[0]
    file_id = file.split("/")[-2]
    RPM[file_id] = rpm

In [ ]:
# Get stage by case id
file = "/home/marcelo/Documents/UTEM/Research/cancer/clinical.cart.2021-04-27/clinical.tsv"
data = pd.read_csv(file, sep='\t', lineterminator='\n')

stages={}
for index, row in data.iterrows():
    stages[row["case_submitter_id"]] = row["tumor_stage"]

In [ ]:
# Get file id by case id
file = "/home/marcelo/Documents/UTEM/Research/cancer/gdc_sample_sheet.2021-04-27.tsv"
data = pd.read_csv(file, sep='\t', lineterminator='\n')

cases={}
for index, row in data.iterrows():
    cases[row["Case ID"]] = row["File ID"]

In [ ]:
# Create a data frame for stage and rpm
stageID,rpmValues=[],[]
for caseID in stages:
    stage = stages[caseID]
    fileID = cases[caseID]
    rpm    = (RPM[fileID])
    rpmValues.append(rpm)
    if stage=='stage i' or stage=='stage ia' or stage=='stage ib':
        stageID.append("I") #stageI.append(rpm)
    elif stage=='stage ii' or stage=='stage iia' or stage=='stage iib':
        stageID.append("II") #stageII.append(rpm)   
    elif stage=='stage iii' or stage=='stage iiia' or stage=='stage iiib' or stage=='stage iiic':
        stageID.append("III") #stageIII.append(rpm)  
    elif stage=='stage iv':
        stageID.append("IV") #stageIV.append(rpm)     
    elif stage=='stage x':
        stageID.append("X") #stageX.append(rpm)  
    else:
        stageID.append("None")
        
dataF = pd.DataFrame( data = {'Stage': stageID, 'RPM': rpmValues})

In [ ]:
#Plot results

sns.set_style("whitegrid")
tips = sns.load_dataset("tips")
ax = sns.swarmplot(x="Stage", y="RPM", data=dataF,order=["I","II","III","IV","X","None"]).set_title("miR-149")

# ax = sns.boxplot(x="Stage", y="RPM", data=dataF,
#         showcaps=False,boxprops={'facecolor':'None'},
#         showfliers=False,whiskerprops={'linewidth':0})


plt.savefig("mir149.svg")

In [ ]:
# Get number of samples per stage

dups_shape = dataF.pivot_table(index=['Stage'], aggfunc='size')
print (dups_shape)